# Recommendation System Backend (Movies + Products)
FastAPI + SQLite backend with movies, products, ratings, and recommendations.


In [ ]:
!pip install fastapi uvicorn sqlalchemy aiosqlite nest_asyncio scikit-learn

In [ ]:

import nest_asyncio
import uvicorn
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

DATABASE_URL = "sqlite:///./recommendation.db"
engine = create_engine(DATABASE_URL, connect_args={"check_same_thread": False})
Base = declarative_base()
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)


In [ ]:

class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, unique=True, index=True)

class Movie(Base):
    __tablename__ = "movies"
    id = Column(Integer, primary_key=True, index=True)
    title = Column(String, index=True)
    genre = Column(String)

class Product(Base):
    __tablename__ = "products"
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    category = Column(String)

class Rating(Base):
    __tablename__ = "ratings"
    id = Column(Integer, primary_key=True, index=True)
    user_id = Column(Integer, ForeignKey("users.id"))
    item_type = Column(String)  # 'movie' or 'product'
    item_id = Column(Integer)
    rating = Column(Float)


In [ ]:

Base.metadata.drop_all(bind=engine)
Base.metadata.create_all(bind=engine)
session = SessionLocal()

# Demo user
demo_user = User(id=1, name="Demo User")
session.add(demo_user)

# Movies
movies = [
    Movie(title="Inception", genre="Sci-Fi"),
    Movie(title="Interstellar", genre="Sci-Fi"),
    Movie(title="The Dark Knight", genre="Action"),
    Movie(title="La La Land", genre="Romance"),
    Movie(title="Titanic", genre="Romance"),
]
session.add_all(movies)

# Products
products = [
    Product(name="iPhone 15", category="Electronics"),
    Product(name="MacBook Pro", category="Electronics"),
    Product(name="Running Shoes", category="Sports"),
    Product(name="Football", category="Sports"),
    Product(name="Coffee Maker", category="Home"),
]
session.add_all(products)

# Ratings
ratings = [
    Rating(user_id=1, item_type="movie", item_id=1, rating=5),
    Rating(user_id=1, item_type="movie", item_id=5, rating=4),
    Rating(user_id=1, item_type="product", item_id=1, rating=5),
    Rating(user_id=1, item_type="product", item_id=3, rating=3),
]
session.add_all(ratings)
session.commit()


In [ ]:

app = FastAPI(title="Recommendation Backend")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


In [ ]:

@app.get("/movies")
def get_movies():
    db = SessionLocal()
    movies = db.query(Movie).all()
    return [{"id": m.id, "title": m.title, "genre": m.genre} for m in movies]

@app.post("/add-movie")
def add_movie(title: str, genre: str):
    db = SessionLocal()
    movie = Movie(title=title, genre=genre)
    db.add(movie)
    db.commit()
    return {"message": "Movie added"}

@app.get("/products")
def get_products():
    db = SessionLocal()
    products = db.query(Product).all()
    return [{"id": p.id, "name": p.name, "category": p.category} for p in products]

@app.post("/add-product")
def add_product(name: str, category: str):
    db = SessionLocal()
    product = Product(name=name, category=category)
    db.add(product)
    db.commit()
    return {"message": "Product added"}

@app.post("/rate")
def rate_item(user_id: int, item_type: str, item_id: int, rating: float):
    db = SessionLocal()
    new_rating = Rating(user_id=user_id, item_type=item_type, item_id=item_id, rating=rating)
    db.add(new_rating)
    db.commit()
    return {"message": "Rating added"}


In [ ]:

def recommend_items(user_id: int, item_type: str):
    db = SessionLocal()
    ratings = pd.read_sql(db.query(Rating).filter(Rating.item_type == item_type).statement, db.bind)
    if ratings.empty:
        return []

    pivot = ratings.pivot_table(index="user_id", columns="item_id", values="rating").fillna(0)
    if user_id not in pivot.index:
        return []

    similarity = cosine_similarity(pivot)
    sim_df = pd.DataFrame(similarity, index=pivot.index, columns=pivot.index)

    similar_users = sim_df[user_id].sort_values(ascending=False).index[1:]
    recommendations = []
    for sim_user in similar_users:
        user_ratings = pivot.loc[sim_user]
        recommended_items = user_ratings[user_ratings > 4].index.tolist()
        recommendations.extend(recommended_items)

    recommendations = list(set(recommendations))

    if item_type == "movie":
        items = db.query(Movie).filter(Movie.id.in_(recommendations)).all()
        return [{"id": i.id, "title": i.title, "genre": i.genre} for i in items]
    else:
        items = db.query(Product).filter(Product.id.in_(recommendations)).all()
        return [{"id": i.id, "name": i.name, "category": i.category} for i in items]

@app.get("/recommend/movies/{user_id}")
def recommend_movies(user_id: int):
    return recommend_items(user_id, "movie")

@app.get("/recommend/products/{user_id}")
def recommend_products(user_id: int):
    return recommend_items(user_id, "product")


In [ ]:

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)
